<a href="https://colab.research.google.com/github/fowardelcac/Titanic_2.0/blob/main/Modelos/Modelado_Dist_algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/fowardelcac/Titanic_2.0.git

Cloning into 'Titanic_2.0'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 121 (delta 41), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (121/121), 155.51 KiB | 7.07 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Funciones

In [3]:
def split(df):
  n = len(df.columns) - 1
  X = df.drop('Survived', axis=1).values.reshape((-1, n))
  y = df.Survived
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.3, random_state = 42)
  return X_train, X_test, y_train, y_test

In [4]:
def modelar(modelo, X_train, y_train):
   return modelo.fit(X_train, y_train) 

In [110]:
def predecir(modelo, X_test, y_test, accuracy_score):
  y_pred = modelo.predict(X_test)
  score = round(accuracy_score(y_test, y_pred), 3)
  return score

def predecir_rdo(modelo, X_test, y_test):
  return modelo.predict(X_test)

In [6]:
def edit_age(df):
  serie = df['Age'].dropna()
  # Obtener la cantidad de valores nulos en la columna 'Age'
  size = df['Age'].isna().sum()
  # Generar una nueva Serie con valores aleatorios de la Serie 'serie'
  random_values = np.random.choice(serie, size=size)
  # Reemplazar los valores NaN en la columna 'Age' con los valores aleatorios
  df.loc[df['Age'].isna(), 'Age'] = random_values
  return df

In [7]:
def onehot(df):
  encoder = OneHotEncoder()
  data = pd.DataFrame(encoder.fit_transform(df['Pclass'].values.reshape(-1, 1)).toarray(), columns=['1', '2', '3'])
  return data

In [8]:
def edit(df):
  df = edit_age(df)

  mapeo = {'male': 0, 'female': 1}
  df.Sex = df.Sex.map(mapeo)
  
  df['familia'] = df.SibSp + df.Parch

  data = onehot(df)
  df.reset_index('PassengerId', inplace=True)
  df = pd.concat([df, data], axis=1).set_index('PassengerId')

  scaler = MinMaxScaler().fit_transform(df['Fare'].values.reshape(-1, 1))
  scaled_series = pd.Series(scaler.flatten(), index=df.index)
  df.Fare = scaled_series

  df.drop(['SibSp', 'Parch', 'Cabin', 'Ticket', 'Name', 'Pclass', 'Embarked'], axis = 1, inplace = True)
  return df   

In [16]:
def modelar_():
  modelos_lista = [RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression()]
  data = create_df_azar(df)
  cols_d = data.columns[:-1]
  
  X_train, X_test, y_train, y_test = split(data)
  dic_m = {}
  for i in modelos_lista:  
    modelo = modelar(i, X_train, y_train)
    acc = predecir(modelo, X_test, y_test, accuracy_score)
    dic_m[i] = [cols_d, acc]
  return dic_m

In [15]:
def random_samples():
  columnas = ['Sex', 'Age', 'Fare', 'familia', '1', '2', '3']
  n_col =  random.randint(1, len(columnas))
  random_c = random.sample(columnas, n_col)
  return random_c

def create_df_azar(df):
  columnas = random_samples()
  data = df.filter(columnas, axis = 1)
  data['Survived'] = df.Survived
  return data

In [103]:
def grid(modelo, params, X_train, y_train):
  grid_rf = GridSearchCV(modelo, params, cv = 5).fit(X_train, y_train)
  mejores_parametros = grid_rf.best_params_
  mejor_score = grid_rf.best_score_

  print("Mejores parámetros:", mejores_parametros)
  print("Mejor score:", mejor_score)

In [9]:
df = pd.read_csv('/content/Titanic_2.0/Datasets_originales/train.csv').set_index('PassengerId')

In [10]:
df = edit(df)
df.head()

,Survived,Sex,Age,Fare,familia,1,2,3
PassengerId,,,,,,,,
1,0,0,22.0,0.014151,1,0.0,0.0,1.0
2,1,1,38.0,0.139136,1,1.0,0.0,0.0
3,1,1,26.0,0.015469,0,0.0,0.0,1.0
4,1,1,35.0,0.103644,1,1.0,0.0,0.0
5,0,0,35.0,0.015713,0,0.0,0.0,1.0


In [11]:
scaler = MinMaxScaler().fit_transform(df['Age'].values.reshape(-1, 1))
scaled_series = pd.Series(scaler.flatten(), index=df.index)
df.Age = scaled_series


In [12]:
X_train, X_test, y_train, y_test = split(df)

In [13]:
modelos_lista = [RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression()]

Sex	Age	Fare	familia	1	2	3 age y fare escaladas



In [14]:
for i in modelos_lista:
  modelo = modelar(i, X_train, y_train)
  acc = predecir(modelo, X_test, y_test, accuracy_score)
  print(f"Modelo: {i}, score:", acc)

Modelo: RandomForestClassifier(), score: 0.792
Modelo: DecisionTreeClassifier(), score: 0.74
Modelo: KNeighborsClassifier(), score: 0.808
Modelo: LogisticRegression(), score: 0.8


In [71]:
lista_dic =[]
for i in range(15):
  lista_dic.append(modelar_())

In [77]:
lista_dic[0].keys()

dict_keys([RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), LogisticRegression()])

In [85]:
scores = []
for diccionario in lista_dic:
    for item in diccionario.values():
        scores.append(item[1])

for index, diccionario in enumerate(lista_dic):
    for clave in diccionario.keys():
        print(index+1, clave, ':', diccionario[clave][1])
        print('*' * 50)

1 RandomForestClassifier() : 0.646
1 DecisionTreeClassifier() : 0.646
1 KNeighborsClassifier() : 0.657
1 LogisticRegression() : 0.715
2 RandomForestClassifier() : 0.788
2 DecisionTreeClassifier() : 0.734
2 KNeighborsClassifier() : 0.801
2 LogisticRegression() : 0.798
3 RandomForestClassifier() : 0.771
3 DecisionTreeClassifier() : 0.769
3 KNeighborsClassifier() : 0.766
3 LogisticRegression() : 0.79
4 RandomForestClassifier() : 0.792
4 DecisionTreeClassifier() : 0.737
4 KNeighborsClassifier() : 0.785
4 LogisticRegression() : 0.792
5 RandomForestClassifier() : 0.652
5 DecisionTreeClassifier() : 0.614
5 KNeighborsClassifier() : 0.684
5 LogisticRegression() : 0.707
6 RandomForestClassifier() : 0.609
6 DecisionTreeClassifier() : 0.609
6 KNeighborsClassifier() : 0.609
6 LogisticRegression() : 0.609
7 RandomForestClassifier() : 0.644
7 DecisionTreeClassifier() : 0.633
7 KNeighborsClassifier() : 0.662
7 LogisticRegression() : 0.668
8 RandomForestClassifier() : 0.758
8 DecisionTreeClassifier() :

In [99]:
lista_dic[10]

{RandomForestClassifier(): [Index(['familia', 'Sex', '3', 'Fare'], dtype='object'),
  0.79],
 DecisionTreeClassifier(): [Index(['familia', 'Sex', '3', 'Fare'], dtype='object'),
  0.768],
 KNeighborsClassifier(): [Index(['familia', 'Sex', '3', 'Fare'], dtype='object'),
  0.792],
 LogisticRegression(): [Index(['familia', 'Sex', '3', 'Fare'], dtype='object'),
  0.801]}


1. RF:  '3', 'Fare', '2', 'Sex', 'Age', 'familia', '1', -> 0.798 / 'familia', 'Sex', '3', 'Fare' -> 0.79
2. Arbol:  '3', 'Fare', '2', 'Sex', 'Age', 'familia', '1', -> 0.756 / 'Sex', '3', -> 0.787
3. KNN:   '3', 'Fare', '2', 'Sex', 'Age', 'familia', '1', -> 0.808 / 'Fare', 'familia', 'Sex', '2', 'Age', '3' -> 0.801
4. LR:   'familia', 'Sex', '3', 'Fare', -> 0.801


In [93]:
warnings.filterwarnings('ignore')

# Random forest

In [100]:
data_rf = df.filter([ 'familia', 'Sex', '3', 'Fare', 'Survived'], axis = 1)
X_train, X_test, y_train, y_test = split(data_rf)

In [101]:
modelo_rf = modelar(RandomForestClassifier(), X_train, y_train)
acc = predecir(modelo_rf, X_test, y_test, accuracy_score)
print(acc)

0.785


In [105]:
parametros = {
    'n_estimators': [100, 200, 300],  # número de árboles en el bosque
    'max_depth': [None, 5, 10],  # profundidad máxima de cada árbol
    'min_samples_split': [2, 5, 10],  # número mínimo de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4]  # número mínimo de muestras requeridas en cada hoja
}

grid(RandomForestClassifier(), parametros, X_train, y_train)

Mejores parámetros: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Mejor score: 0.8167714884696016


In [106]:
modelo_rf = modelar(RandomForestClassifier(max_depth =  5, min_samples_leaf = 2, min_samples_split = 5, n_estimators = 300), X_train, y_train)
acc = predecir(modelo_rf, X_test, y_test, accuracy_score)
print(acc)

0.801


# Arbol de desicion

In [117]:
data_a = df.filter([ '3', 'Fare', '2', 'Sex', 'Age', 'familia', '1', 'Survived'], axis = 1)
X_train, X_test, y_train, y_test = split(data_a)
modelo_tree = modelar(DecisionTreeClassifier(), X_train, y_train)
acc = predecir(modelo_tree, X_test, y_test, accuracy_score)
print(acc)

0.752


In [118]:
parametros = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid(DecisionTreeClassifier(), parametros, X_train, y_train)

Mejores parámetros: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mejor score: 0.8241788958770091


In [119]:
modelo_tree = modelar(DecisionTreeClassifier(criterion = 'entropy', max_depth = 5, min_samples_leaf = 2, min_samples_split= 2), X_train, y_train)
acc = predecir(modelo_tree, X_test, y_test, accuracy_score)
print(acc)

0.812
